# **MIP1_Run3**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """ You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened.
Please formulate this as a mathematical optimization model."""

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:

Let's start with defining the parameters:
- Number of customers -> I = {1, 2, ..., n}
- Number of sites for potential new facilities -> J = {1, 2, ..., m}
- Fixed yearly cost for establishing a facility -> Fj (for j in set of potential facilities J)
- Maximum annual service volume for each facility -> Sj (for j in set of potential facilities J)
- Demand for each customer -> Di (for i in set of customers I)
- Transportation cost for servicing a specific customer from a specific facility -> Tij (for each i in I and j in J)

Decision Variables:

Next, define binary decision variables:
- xj, equals 1 if a facility is established at location j, 0 otherwise (for j in set J)
- yij, equals 1 if customer i is assigned to facility j, 0 otherwise (for each i in I, and j in J)

Objective Function:

The objective is to minimize fixed costs for establishing facilities and transportation costs. The objective function can be stated as:
   
    min ∑ (Fj * xj) + ∑ ∑ (Tij * yij)
    (j in J

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [ ]:
Markdown(response2.choices[0].message.content)

Now let's translate the optimization problem discussed above into python code using the Pyomo library. 

First, we need to install the pyomo library. You can install it via pip:

```python
pip install pyomo
```

Now, implement the optimization problem.
```python

# Import required libraries
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define index sets (sample data)
I = list(range(1, 11))       # Customers
J = list(range(1, 6))        # Potential facilities

# Define parameters (using random values as sample data)

# Fixed yearly cost for establishing a facility
Fj = {j: 10 * j for j in J}

# Maximum annual service volume for each facility
Sj = {j: 15 * j for j in J}

# Demand for each customer 
Di = {i: i for i in I} 

# Transportation cost for servicing a customer from a specific facility
Tij = {(i, j): i * j for i in I for j in J}

# Define decision variables
model.x = Var(J, within=Binary)
model.y = Var(I, J, within=Binary)

# Define objective function
model.obj = Objective(
    expr=sum(Fj[j] * model.x[j] for j in J) + 
    sum(Tij[(i,j)] * model.y[i,j] for i in I for j in J),
    sense=minimize)

# Define constraints

# Each customer demand must be met
model.demand = ConstraintList()
for i in I:
    model.demand.add(sum(model.y[i,j] for j in J) == 1)

# The maximum service volume for each facility cannot be exceeded
model.service = ConstraintList()
for j in J:
    model.service.add(sum(Di[i] * model.y[i,j] for i in I) - Sj[j] * model.x[j] <= 0)

# A customer can only be served by a facility that is opened
model.assignment = ConstraintList()
for i in I:
    for j in J:
        model.assignment.add(model.y[i,j] - model.x[j] <= 0)

# Solve model
solver = SolverFactory('glpk')
solver.solve(model)

# Print result (only the decision variables)
print("Solution: ")
for j in J:
    print(f"Establish facility at location {j}: {model.x[j].value}")
for i in I:
    for j in J:
        print(f"Assign customer {i} to facility {j}: {model.y[i,j].value}")
```

Please note that depending on the version of Glpk solver installed in your system, you may get different results. The 'glpk' solver used in this solution may or may not be available by default. If not, it needs to be installed separately.

On running the code, the solution prints whether or not to establish a facility at each potential location and which customers should be assigned to which facilities (given that a facility is established there) to minimize total costs. The result may vary as the sample data used in this solution is random.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define index sets (sample data)
I = list(range(1, 11))       # Customers
J = list(range(1, 6))        # Potential facilities

# Define parameters (using random values as sample data)

# Fixed yearly cost for establishing a facility
Fj = {j: 10 * j for j in J}

# Maximum annual service volume for each facility
Sj = {j: 15 * j for j in J}

# Demand for each customer
Di = {i: i for i in I}

# Transportation cost for servicing a customer from a specific facility
Tij = {(i, j): i * j for i in I for j in J}

# Define decision variables
model.x = Var(J, within=Binary)
model.y = Var(I, J, within=Binary)

# Define objective function
model.obj = Objective(
    expr=sum(Fj[j] * model.x[j] for j in J) +
    sum(Tij[(i,j)] * model.y[i,j] for i in I for j in J),
    sense=minimize)

# Define constraints

# Each customer demand must be met
model.demand = ConstraintList()
for i in I:
    model.demand.add(sum(model.y[i,j] for j in J) == 1)

# The maximum service volume for each facility cannot be exceeded
model.service = ConstraintList()
for j in J:
    model.service.add(sum(Di[i] * model.y[i,j] for i in I) - Sj[j] * model.x[j] <= 0)

# A customer can only be served by a facility that is opened
model.assignment = ConstraintList()
for i in I:
    for j in J:
        model.assignment.add(model.y[i,j] - model.x[j] <= 0)

# Solve model
solver = SolverFactory('glpk')
solver.solve(model)

# Print result (only the decision variables)
print("Solution: ")
for j in J:
    print(f"Establish facility at location {j}: {model.x[j].value}")
for i in I:
    for j in J:
        print(f"Assign customer {i} to facility {j}: {model.y[i,j].value}")

Solution: 
Establish facility at location 1: 1.0
Establish facility at location 2: 1.0
Establish facility at location 3: 1.0
Establish facility at location 4: 0.0
Establish facility at location 5: 0.0
Assign customer 1 to facility 1: 1.0
Assign customer 1 to facility 2: 0.0
Assign customer 1 to facility 3: 0.0
Assign customer 1 to facility 4: 0.0
Assign customer 1 to facility 5: 0.0
Assign customer 2 to facility 1: 0.0
Assign customer 2 to facility 2: 0.0
Assign customer 2 to facility 3: 1.0
Assign customer 2 to facility 4: 0.0
Assign customer 2 to facility 5: 0.0
Assign customer 3 to facility 1: 0.0
Assign customer 3 to facility 2: 1.0
Assign customer 3 to facility 3: 0.0
Assign customer 3 to facility 4: 0.0
Assign customer 3 to facility 5: 0.0
Assign customer 4 to facility 1: 1.0
Assign customer 4 to facility 2: 0.0
Assign customer 4 to facility 3: 0.0
Assign customer 4 to facility 4: 0.0
Assign customer 4 to facility 5: 0.0
Assign customer 5 to facility 1: 0.0
Assign customer 5 to f

In [ ]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()
#data inputted by human
# Demand for each customer
Di = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}
# Maximum annual service volume for each facility
Sj = {1: 500, 2: 500, 3: 500}
# Define index sets (sample data)
I = list(Di.keys())       # Customers
J = list(Sj.keys())        # Potential facilities

# Define parameters (using random values as sample data)

# Fixed yearly cost for establishing a facility
Fj = {1: 1000, 2: 1000, 3: 1000}




# Transportation cost for servicing a customer from a specific facility
Tij = {
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (4, 1): 8,  (4, 2): 5,  (4, 3): 3,
    (5, 1): 10, (5, 2): 8,  (5, 3): 4,
}
#end

# Define decision variables
model.x = Var(J, within=Binary)
model.y = Var(I, J, within=Binary)

# Define objective function
model.obj = Objective(
    expr=sum(Fj[j] * model.x[j] for j in J) +
    sum(Tij[(i,j)] * model.y[i,j] for i in I for j in J),
    sense=minimize)

# Define constraints

# Each customer demand must be met
model.demand = ConstraintList()
for i in I:
    model.demand.add(sum(model.y[i,j] for j in J) == 1)

# The maximum service volume for each facility cannot be exceeded
model.service = ConstraintList()
for j in J:
    model.service.add(sum(Di[i] * model.y[i,j] for i in I) - Sj[j] * model.x[j] <= 0)

# A customer can only be served by a facility that is opened
model.assignment = ConstraintList()
for i in I:
    for j in J:
        model.assignment.add(model.y[i,j] - model.x[j] <= 0)

# Solve model
solver = SolverFactory('glpk')
solver.solve(model)

# Print result (only the decision variables)
print(model.obj())
print("Solution: ")
for j in J:
    print(f"Establish facility at location {j}: {model.x[j].value}")
for i in I:
    for j in J:
        print(f"Assign customer {i} to facility {j}: {model.y[i,j].value}")

2025.0
Solution: 
Establish facility at location 1: 0.0
Establish facility at location 2: 1.0
Establish facility at location 3: 1.0
Assign customer 1 to facility 1: 0.0
Assign customer 1 to facility 2: 1.0
Assign customer 1 to facility 3: 0.0
Assign customer 2 to facility 1: 0.0
Assign customer 2 to facility 2: 0.0
Assign customer 2 to facility 3: 1.0
Assign customer 3 to facility 1: 0.0
Assign customer 3 to facility 2: 1.0
Assign customer 3 to facility 3: 0.0
Assign customer 4 to facility 1: 0.0
Assign customer 4 to facility 2: 1.0
Assign customer 4 to facility 3: 0.0
Assign customer 5 to facility 1: 0.0
Assign customer 5 to facility 2: 0.0
Assign customer 5 to facility 3: 1.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**